In [1]:
import eventDrivenSimulation as eds
import simpy
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import show as sh
import requests
import folium
import haversine
import math
import bibliothek as bib
import random

### Setup of a simulation

UE-Data is already generated or loaded & comp dataframe is generated

In [2]:
df=pd.read_csv('ue_data_pandas')

* Define parameters

In [3]:
max_prb=50 #number of prbs defined by the system bandwidth
ue_nr=15 #number of ues per pci
metric=[1,1] #defines the exponents of the metric (standard metric is pf ->[1,1])
cluster=[133,775]

* choose a mode to get UEs from the UE-list r='random' (randomly selected),'determininstic' (first N user selected)
* Define simulation environment
* get users structured in a dictionary out of df (ue_dict)
* get index of the users that will be selected

In [4]:
#r='random'
r='deterministic'
env=simpy.Environment()
ue_dict=eds.df_to_ue_lists(df,cluster,6,env)
index=eds.select_user_index(r, ue_nr, ue_dict, cluster)
print(index)

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]]


* select the users from the defined index for each tp (ue_per_tp)
* all users selcted for the simulation in one array (ue_all)
* Define PRB number for comp user for different users for the cluster (dict that contains TP: prb_number)

In [5]:
ue_per_tp,ue_all=eds.get_user_from_cluster(ue_dict,cluster,ue_nr,index)
prb_number_comp=eds.calculate_prb_number_comp(ue_all,cluster,max_prb,ue_nr)

* Define the schedulers (central scheduler & 1 scheduler for each TP)

In [6]:
sched_l=[]
sched_central=eds.sched_inst(env) #central scheduler
for i in cluster:
    sched_l.append(eds.sched_inst(env))

* Define timer for scheduling events and packet arrival rate

In [7]:
SCHEDULE_T=2
mu=4

* select the users that will use comp (ue_comp)
* select the users that will NOT use comp (ue_noComp)

In [8]:
ue_noCoMP,ue_comp=eds.seperate_comp_noCoMP(cluster,ue_per_tp)

* Initialize the users that were selcted for the EDS and start the packet arrivals
* Start the process for the central scheduler
* Start the process for the other schedulers
* run the simulation till the end of the timer

In [9]:
ue_comp

array([<eventDrivenSimulation.ue object at 0x0000026A5AFDDA30>,
      dtype=object)

In [10]:
for j in ue_all:
    env.process(j.best_effort_stat(env,mu))

env.process(sched_central.central_scheduler(env,ue_comp,SCHEDULE_T,cluster,prb_number_comp,metric,'phaseshift'))

counter=0
for i in cluster:
    ue_list=ue_noCoMP[i]
    ue_sep=ue_all[counter*ue_nr:((counter+1)*ue_nr)]
    prb_number_normal=max_prb-prb_number_comp[i]
    env.process(sched_l[counter].scheduler(env,ue_sep,SCHEDULE_T,cluster,max_prb,ue_list,prb_number_normal,metric))
    counter=counter+1
timer=2000
env.run(until=timer)

C:\Users\Caroline Zoll\EDS\eventDrivenSimulation.py:240: RuntimeWarning: divide by zero encountered in double_scalars
  metric=np.append(metric,((i.cp)**e1/(i.mR**e2)))
C:\Users\Caroline Zoll\EDS\eventDrivenSimulation.py:270: RuntimeWarning: divide by zero encountered in double_scalars
  metric=np.append(metric,((cp)**e1/(i.mR2)**e2))


In [11]:
m=0
for i in ue_all:
    m+=i.mR2/i.mR
print(m/len(ue_all))

1.0630085428640017


### All in one 

In [12]:
#r='random'
r='deterministic'
env=simpy.Environment()
ue_dict=eds.df_to_ue_lists(df,cluster,6,env)
index=eds.select_user_index(r, ue_nr, ue_dict, cluster)

ue_per_tp,ue_all=eds.get_user_from_cluster(ue_dict,cluster,ue_nr,index)
prb_number_comp=eds.calculate_prb_number_comp(ue_all,cluster,max_prb,ue_nr)

sched_l=[]
sched_central=eds.sched_inst(env) #central scheduler
for i in cluster:
    sched_l.append(eds.sched_inst(env))
    
SCHEDULE_T=2
mu=4

ue_noCoMP,ue_comp=eds.seperate_comp_noCoMP(cluster,ue_per_tp)

for j in ue_all:
    env.process(j.best_effort_stat(env,mu))

env.process(sched_central.central_scheduler(env,ue_comp,SCHEDULE_T,cluster,prb_number_comp,metric,'phaseshift'))

counter=0
for i in cluster:
    ue_list=ue_noCoMP[i]
    ue_sep=ue_all[counter*ue_nr:((counter+1)*ue_nr)]
    prb_number_normal=max_prb-prb_number_comp[i]
    env.process(sched_l[counter].scheduler(env,ue_sep,SCHEDULE_T,cluster,max_prb,ue_list,prb_number_normal,metric))
    counter=counter+1
timer=2000
env.run(until=timer)

In [13]:
m=0
for i in ue_all:
    m+=i.mR2/i.mR
print(m/len(ue_all))

1.0660845268861363
